In [1]:
import os 
import json 
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY =os.getenv("OPENAI_API_KEY")

In [246]:
import requests
from newspaper import Article

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
}

article_urls = [
    "https://www.artificialintelligence-news.com/2023/04/21/google-creates-new-ai-division-to-challenge-openai/",
    "https://www.artificialintelligence-news.com/2022/07/27/openai-gpt-3-is-a-convincing-philosopher/",
    "https://www.artificialintelligence-news.com/2022/01/25/meta-claims-new-ai-supercomputer-will-set-records/"
]

session = requests.Session()
pages_content = []

for url in article_urls:
    try:
        response = session.get(url, headers=headers, timeout=10)
        
        if response.status_code == 200:
            article = Article(url)
            article.download()
            article.parse()
            
            print(f"Title: {article.title}")
            print(f"Text: {article.text}")
            
            pages_content.append({"url": url, "title": article.title, "text": article.text})
        else:
            print(f"Failed to fetch article at {url}")
    except Exception as e:
        print(f"Error occurred while fetching article at {url}: {e}")

print("\nAll page contents:")
print(pages_content)


Title: Google creates new AI division to challenge OpenAI
Text: Ryan is a senior editor at TechForge Media with over a decade of experience covering the latest technology and interviewing leading industry figures. He can often be sighted at tech conferences with a strong coffee in one hand and a laptop in the other. If it's geeky, he’s probably into it. Find him on Twitter (@Gadget_Ry) or Mastodon (@gadgetry@techhub.social)

Google has consolidated its AI research labs, Google Brain and DeepMind, into a new unit named Google DeepMind.

The move is seen as a strategic way for Google to maintain its edge in the competitive AI industry and compete with OpenAI. By combining the talent and resources of both entities, Google DeepMind aims to accelerate AI advancements while maintaining ethical standards.

The new unit will be responsible for spearheading groundbreaking AI products and advancements, and it will work closely with other Google product areas to deliver AI research and products.


In [197]:
import os

def save_pages_content_to_text_file(pages_content, filename='pages_content.txt'):
    current_directory = os.getcwd()
    file_path = os.path.join(current_directory, filename)
    
    with open(file_path, 'w', encoding='utf-8') as file:
        for page in pages_content:
            file.write(f"URL: {page['url']}\n")
            file.write(f"Title: {page['title']}\n")
            file.write("Text:\n")
            file.write(f"{page['text']}\n\n")
            file.write("=" * 80 + "\n\n")

# After fetching the articles and populating the pages_content list, call the function
save_pages_content_to_text_file(pages_content)


In [198]:
from langchain.document_loaders import UnstructuredFileLoader

In [199]:
loader = UnstructuredFileLoader('pages_content.txt')

In [200]:
docs = loader.load()

In [ ]:
docs[0]

In [67]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

In [7]:
from langchain.chat_models import ChatOpenAI
llm = OpenAI(model_name='gpt-3.5-turbo', 
             temperature=0, 
             max_tokens = 256)

/home/cloudsuperadmin/.local/lib/python3.9/site-packages/langchain/llms/openai.py:165: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/cloudsuperadmin/.local/lib/python3.9/site-packages/langchain/llms/openai.py:677: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [202]:
text_splitter = CharacterTextSplitter()

In [203]:
# load the doc
with open('pages_content.txt') as f:
    news = f.read()
texts = text_splitter.split_text(news)

In [204]:
len(texts)

3

In [205]:
from langchain.docstore.document import Document

docs = [Document(page_content=t) for t in texts[:4]]

In [ ]:
docs

In [207]:
from langchain.chains.summarize import load_summarize_chain
import textwrap

In [239]:
fact_extraction_prompt = PromptTemplate(
    input_variables=["text_input"],
    template="Extract the key facts out of texts. Don't include opinions. \
    Summarize the news :\n\n {text_input}"
)

In [240]:
fact_extraction_chain = LLMChain(llm=llm, prompt=fact_extraction_prompt)

facts = fact_extraction_chain.run(docs)

wrapped_text = textwrap.fill(facts, 
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(wrapped_text)

Google has consolidated its AI research labs, Google Brain and DeepMind, into a new unit named Google DeepMind to maintain its edge in the competitive AI industry and compete with OpenAI. The new unit will be responsible for spearheading groundbreaking AI products and advancements, and it will work closely with other Google product areas to deliver AI research and products. OpenAI’s GPT-3 is capable of being indistinguishable from a human philosopher, according to a study by Eric Schwitzgebel, Anna Strasser, and Matthew Crosby. Meta (formerly Facebook) has unveiled an AI supercomputer called the AI Research SuperCluster (RSC) that it claims will be the world’s fastest. RSC is set to be fully built in mid-2022 and is estimated to be 20x faster than Meta’s current V100-based clusters.


In [242]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate, LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [243]:
chatGPT = ChatOpenAI(temperature=0)

In [244]:
messages = [
    SystemMessage(content="You are an expert at making strong factual summarizations.\
     you have to provide: urls: {url}, titles: {article.title}, texts: {article.text} of the given content:{page_content} and give a summaries for all the links provided "),
    HumanMessage(content=wrapped_text)
]
responses = chatGPT(messages)

In [245]:
results = textwrap.fill(responses.content, 
                             width=1000,
                             break_long_words=False,
                             replace_whitespace=False)
print(results)

URLs: 
- https://www.business-standard.com/article/technology/google-consolidates-ai-research-labs-into-new-unit-called-google-deepmind-122121300764_1.html
- https://www.analyticsinsight.net/openais-gpt-3-capable-of-being-indistinguishable-from-human-philosopher-study/
- https://www.zdnet.com/article/meta-unveils-ai-supercomputer-aiming-to-be-worlds-fastest/

Titles:
- Google consolidates AI research labs into new unit called Google DeepMind
- OpenAI’s GPT-3 Capable of Being Indistinguishable from Human Philosopher: Study
- Meta unveils AI supercomputer aiming to be world's fastest

Texts:
- Google has merged its AI research labs, Google Brain and DeepMind, into a new unit called Google DeepMind. The new unit will be responsible for spearheading groundbreaking AI products and advancements, and it will work closely with other Google product areas to deliver AI research and products. The move is aimed at maintaining Google's edge in the competitive AI industry and competing with OpenAI.
